In [20]:
import os
import re
from tqdm import tqdm
from datetime import datetime
import xlrd
import xlutils.copy
import xlwt
from xlwt import Workbook
from copy import deepcopy
import glob

import pandas as pd
import numpy as np
import camelot
import tabula
from pyexcel.cookbook import merge_all_to_a_book

In [10]:
def _getOutCell(outSheet, colIndex, rowIndex):
    """ HACK: Extract the internal xlwt cell representation. """
    row = outSheet._Worksheet__rows.get(rowIndex)
    if not row: return None

    cell = row._Row__cells.get(colIndex)
    return cell

In [11]:
def setOutCell(outSheet, col, row, value):
    """ Change cell value without changing formatting. """
    # HACK to retain cell style.
    previousCell = _getOutCell(outSheet, col, row)
    # END HACK, PART I

    outSheet.write(row, col, value)

    # HACK, PART II
    if previousCell:
        newCell = _getOutCell(outSheet, col, row)
        if newCell:
            newCell.xf_idx = previousCell.xf_idx

In [12]:
def calc_results (df_peak, compound, average_area, constant_1, constant_2, unit):
    base_rrt = float(df_peak['Ret. Time'][df_peak['Name'].str.contains(compound, flags = re.IGNORECASE)].values.tolist()[0])
    rrt_master = []
    impurity_master = []
    rrf_master = []
    for index, row in df_peak.iterrows():
        name =row[1]
        if(name.lower() == compound.lower()):
            impurity_master.append(0)
            rrt_master.append(1)
            rrf_master.append(0)
            continue
        if(name == np.nan or name == ''):
            continue
#             impurity_master.append(0)
#             rrt_master.append(0)
            
        area = float(row[3])
        rrf_cond_1 = df_rrf['Compound'].str.contains(compound, flags = re.IGNORECASE)
        rrf_cond_2 = df_rrf['Impurity/Active Name'].str.contains(name, flags = re.IGNORECASE)
        rrf = df_rrf['RRF'][rrf_cond_1 & rrf_cond_2].values.tolist()
        rrt = float(row[2])
        
        if(not(rrf)):
            print("ignoring {}".format(name))
            impurity_master.append(0)
            rrt_res = round(rrt/base_rrt, ndigits=2)
            rrt_master.append(rrt_res)
            rrf_master.append(0)
            continue
            
        rrf = float(rrf[0])
        impurity = round((area/average_area) * constant_1 * constant_2 * (unit/rrf), ndigits=2)
        rrt_res = round(rrt/base_rrt, ndigits=2)
        impurity_master.append(impurity)
        rrt_master.append(rrt_res)
        rrf_master.append(rrf)

    return impurity_master, rrt_master, rrf_master

In [13]:
def shift_row_to_top(df, index_to_shift):
    idx = df.index.tolist()
    idx.remove(index_to_shift)
    df = df.reindex([index_to_shift] + idx)
    return df

In [14]:
def table_extratcor(tables, headers):
    df_result_table =''
    result_tables = []
    for table in tables:
        df_table = table.df
        search = df_table.where(df_table==headers[0]).dropna(how='all').dropna(axis=1)
        inx = list(search.index)
        if(inx):
            inx= inx[0]
            new_header = df_table.iloc[inx]
            new_start_inx = inx+1
            df_table = df_table[new_start_inx:] 
            df_table.columns = new_header
            df_table = df_table[headers]
            result_tables.append(df_table)
        else:
            continue
    df_result_table = pd.concat(result_tables, ignore_index=True)
    return df_result_table

In [15]:
def fill_rs_sheet(output_sheet, df_area_table, df_peak_table, sample_input_list):
    average_area = float(df_area_table["Area"][df_area_table["Title"] == "Average"].values.tolist()[0])
    area_input = list(df_area_table['Area'])
    
    #poject name
    setOutCell(output_sheet, 2, 3, '')
    #Date
    setOutCell(output_sheet, 2, 4, '')
    #Method
    setOutCell(output_sheet, 2, 5, '')
    # WS ID No.
    setOutCell(output_sheet, 1, 9, '')
    # potency
    setOutCell(output_sheet, 3, 9, input_list[-1])
    # use before date
    setOutCell(output_sheet, 5, 9, '')
    # Average area
    setOutCell(output_sheet, 7, 9, average_area)
    # std_wt
    setOutCell(output_sheet, 2, 10, input_list[0])
    #  v1
    setOutCell(output_sheet, 2, 11, input_list[1])
    # v2
    setOutCell(output_sheet, 4, 10, input_list[2])
    #  v3
    setOutCell(output_sheet, 4, 11, input_list[3])
    #  v4
    setOutCell(output_sheet, 6, 10,  input_list[4])
    # v5
    setOutCell(output_sheet, 6, 11, input_list[5])
    # v6
    setOutCell(output_sheet, 8, 10, input_list[6])
    # v7
    setOutCell(output_sheet, 8, 11, input_list[7])
    # factor
    setOutCell(output_sheet, 9, 10, input_list[8])
    # factor
    setOutCell(output_sheet, 9, 11, input_list[9])
    # AR NO
    setOutCell(output_sheet, 1, 14, '')
    # Batch NO
    setOutCell(output_sheet, 3, 14, '')
    # Condition
    setOutCell(output_sheet, 4, 14, '')
    # Label Claim
    setOutCell(output_sheet, 5, 14, sample_input_list[8])
    # per unit
    setOutCell(output_sheet, 7, 14, sample_input_list[9])
    # sample_wt
    setOutCell(output_sheet, 2, 15, sample_input_list[0])
    #  v1
    setOutCell(output_sheet, 2, 16, sample_input_list[1])
    # v2
    setOutCell(output_sheet, 4, 15, sample_input_list[2])
    #  v3
    setOutCell(output_sheet, 4, 16, sample_input_list[3])
    #  v4
    setOutCell(output_sheet, 6, 15, sample_input_list[4])
    # v5
    setOutCell(output_sheet, 6, 16, sample_input_list[5])
    # v6
    setOutCell(output_sheet, 8, 15, sample_input_list[6])
    # v7
    setOutCell(output_sheet, 8, 16, sample_input_list[7])
#     areas
    setOutCell(output_sheet, 12, 5, area_input[0])
    setOutCell(output_sheet, 12, 6, area_input[1])
    setOutCell(output_sheet, 12, 7, area_input[2])
    setOutCell(output_sheet, 12, 8, area_input[3])
    setOutCell(output_sheet, 12, 9, area_input[4])
    setOutCell(output_sheet, 12, 10, area_input[5])
    setOutCell(output_sheet, 12, 11, area_input[6])
    setOutCell(output_sheet, 12, 12, area_input[7])
    setOutCell(output_sheet, 12, 13, area_input[8])
#   Impurity table  
    table_row = 20
    for index, row in df_peak_table.iterrows():
        if(table_row > 60):
            break
        setOutCell(output_sheet, 1, table_row, row[0])
        setOutCell(output_sheet, 2, table_row, row[1])
        setOutCell(output_sheet, 3, table_row, row[2])
        setOutCell(output_sheet, 4, table_row, row[3])
        setOutCell(output_sheet, 5, table_row, row[4])
        setOutCell(output_sheet, 6, table_row, row[5])
        table_row +=1

    sum_of_impurities = round(df_peak_table["% w/w"].sum(), ndigits=2)
    setOutCell(output_sheet, 6, 62, sum_of_impurities)


In [16]:
def initiate_report_creation(compound, input_lits):
    sample_wt = df_sample_prep['vials'][df_sample_prep["Compound"].str.contains(compound, flags = re.IGNORECASE)].values.tolist()[0]
    sample_v1 = df_sample_prep['v1'][df_sample_prep["Compound"].str.contains(compound, flags = re.IGNORECASE)].values.tolist()[0]
    sample_v2 = df_sample_prep['v2'][df_sample_prep["Compound"].str.contains(compound, flags = re.IGNORECASE)].values.tolist()[0]
    sample_v3 = df_sample_prep['v3'][df_sample_prep["Compound"].str.contains(compound, flags = re.IGNORECASE)].values.tolist()[0]
    sample_v4 = df_sample_prep['v4'][df_sample_prep["Compound"].str.contains(compound, flags = re.IGNORECASE)].values.tolist()[0]
    sample_v5 = df_sample_prep['v5'][df_sample_prep["Compound"].str.contains(compound, flags = re.IGNORECASE)].values.tolist()[0]
    sample_v6 = df_sample_prep['v6'][df_sample_prep["Compound"].str.contains(compound, flags = re.IGNORECASE)].values.tolist()[0]
    sample_v7 = df_sample_prep['v7'][df_sample_prep["Compound"].str.contains(compound, flags = re.IGNORECASE)].values.tolist()[0]
    label_claim = df_sample_prep['label claim'][df_sample_prep["Compound"].str.contains(compound, flags = re.IGNORECASE)].values.tolist()[0]
    unit = df_sample_prep['per unit'][df_sample_prep["Compound"].str.contains(compound, flags = re.IGNORECASE)].values.tolist()[0]

    constant_1 = (input_list[0]/input_list[1]) * (input_list[2]/input_list[3]) * (input_list[4]/input_list[5])*(input_list[6]/input_list[7]) * (input_list[8]/input_list[9])
    constant_2 = (sample_v1/sample_wt) * (sample_v3/sample_v2) * (sample_v5/sample_v4) * (sample_v7/sample_v6) * (input_list[10]/label_claim)

    # peak table extratcion
    tables = camelot.read_pdf(chrom_input, pages= 'all',flavor='stream')
    df_peak_table = table_extratcor(tables, chrom_headers)
    inx_to_shift = df_peak_table[df_peak_table["Name"].str.contains(compound, flags = re.IGNORECASE)].index[0]
    df_peak_table = shift_row_to_top(df_peak_table, inx_to_shift)
    cond_1 = df_peak_table["Name"] == ''
    cond_2 = df_peak_table["Name"] == np.nan
    inxs_to_remove = df_peak_table[cond_1 | cond_2].index
    df_peak_table = df_peak_table.drop(inxs_to_remove)

    # area table extraction
    tables = camelot.read_pdf(area_input, pages= 'all',flavor='stream')
    df_area_table = table_extratcor(tables, area_headers)
    df_area_table = df_area_table[['Title', 'Area']]
    average_area = float(df_area_table["Area"][df_area_table["Title"] == "Average"].values.tolist()[0])

    # impurity calculation
    impurities, rrts, rrfs = calc_results(df_peak_table, compound, average_area, constant_1, constant_2, unit)
    df_peak_table['RRT'] = rrts
    df_peak_table['RRF'] = rrfs
    df_peak_table["% w/w"] = impurities
    df_peak_table = df_peak_table[['Name', 'Ret. Time','RRT', 'RRF', 'Area', '% w/w']]

    # writing to output sheet
    sample_input_list = [sample_wt, sample_v1, sample_v2, sample_v3, sample_v4, sample_v5, sample_v6, sample_v7, label_claim, unit]
    fill_rs_sheet(rs_template_sheet, df_area_table, df_peak_table, sample_input_list)

    

In [17]:
# Bumetanide
# Acyclovir
# Famotidine
# ketorolacTromethamine
# LabetalolHCl
# input_list = [20.48,20,1,100,5,20,1,1,1,1,99.0]

# compound = input("Enter the compund name [As mentioned in the chromatogram] ")
compound = 'Acyclovir'
input_list = [50.43,100,5,50,5,50,1,1,1,1,94.4]
# input data sources
df_rrf = pd.read_excel(os.path.join(os.getcwd(), 'data', 'Templates', 'RRF-template.xlsx'))
df_sample_prep = pd.read_excel(os.path.join(os.getcwd(), 'data', 'Templates', 'Sample Preparation.xlsx'))
chrom_input = os.path.join(os.getcwd(), "data", "chromatograms", "{}-chromatogram.pdf".format(compound))
area_input = os.path.join(os.getcwd(), "data", "Areas", "{}-areas.pdf".format(compound))

# input_list = [0]*11
# input_list[0] = float(input("Enter the Weight taken "))
# input_list[1] = float(input("Enter the standard preparation v1 ")) 
# input_list[2] = float(input("Enter the standard preparation v2 ")) 
# input_list[3] = float(input("Enter the standard preparation v3 ")) 
# input_list[4] = float(input("Enter the standard preparation v4 ")) 
# input_list[5] = float(input("Enter the standard preparation v5 ")) 
# input_list[6] = float(input("Enter the standard preparation v6 ")) 
# input_list[7] = float(input("Enter the standard preparation v7 ")) 
# input_list[8] = float(input("Enter the standard preparation factor 1 ")) 
# input_list[9] = float(input("Enter the standard preparation factor 2 ")) 
# input_list[10] = float(input("Enter the standard preparation Potency ")) 
 

chrom_headers = ['Peak#','Name','Ret. Time','Area','Area%','RRT']
area_headers = ['Title', 'Ret. Time', 'Area', 'Area%', 'NTP', 'Tailing Factor']



# Output sheet
rs_template_input = xlrd.open_workbook(os.path.join(os.getcwd(), "data", "Templates",'RS-template.xls'), formatting_info=True)
rs_template = xlutils.copy.copy(rs_template_input)
rs_template_sheet = rs_template.get_sheet(0)

initiate_report_creation(compound, input_list)
rs_template.save(os.path.join(os.getcwd(), "data", 'output', '{}-RS.xls'.format(compound)))
print("Report saved successfully, check Output folder.")

ignoring Impurity-B
Report saved successfully, check Output folder.


In [66]:
headers = ['Name','Ret. Time','Area']
headers = ['Area']

In [67]:
chrom_input = os.path.join(os.getcwd(), "data", "chromatograms", "Famotidine-chromatogram.pdf".format(compound))
area_input = os.path.join(os.getcwd(), "data", "areas", "Acyclovir-areas.pdf".format(compound))
tables = camelot.read_pdf(chrom_input, pages= 'all',flavor='stream')

In [68]:
df_result_table =''
result_tables = []
for table in tables:
    df_table = table.df
    search = df_table.where(df_table==headers[0]).dropna(how='all').dropna(axis=1)
    inx = list(search.index)
    if(inx):
        inx= inx[0]
        new_header = df_table.iloc[inx]
        new_start_inx = inx+1
        df_table = df_table[new_start_inx:] 
        df_table.columns = new_header
        df_table = df_table[headers]
        result_tables.append(df_table)
    else:
        continue
df_result_table = pd.concat(result_tables, ignore_index=True)

In [57]:
df_result_table = df_result_table.drop_duplicates(keep='first')
df_peak_table = df_result_table
inx_to_shift = df_peak_table[df_peak_table["Name"].str.contains(compound, flags = re.IGNORECASE)].index[0]
df_peak_table = shift_row_to_top(df_peak_table, inx_to_shift)
cond_1 = df_peak_table["Name"] == ''
cond_2 = df_peak_table["Name"] == np.nan
inxs_to_remove = df_peak_table[cond_1 | cond_2].index
df_peak_table = df_peak_table.drop(inxs_to_remove)


In [58]:
df_peak_table

,Name,Ret. Time,Area
5,Acyclovir,9.77,39871444
0,Unknown,2.84,41952
1,Unknown,3.09,105750
2,Impurity-B,5.80,164175
3,Unknown,6.88,25059
4,Unknown,8.41,8307
6,Unknown,15.04,6044
7,Unknown,15.66,90859
8,Unknown,20.13,2820
9,Unknown,20.26,2573
